In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from joblib import dump, load

pd.options.display.width = 1000 #défaut = 50
pd.options.display.max_columns = 50 #défaut = 20
pd.options.display.max_rows = 100 #défaut = 60
pd.options.display.max_colwidth = 100 #défaut = 50 caractères visualisés dans une colonne

In [3]:
# ****** import des données rééquilibrées par méthode SMOTE() mais non normalisées par méthode StandardScaler() *****

# Import des données training accident: data
# X_sm = pd.read_csv('X_sm.zip', nrows=30000)
X_sm = pd.read_csv('X_sm.zip', nrows=100000)
display(X_sm)

# Import des données training accident: target
y_sm = np.array(pd.read_csv('y_sm.zip', nrows=100000)).ravel()
display(y_sm)

# Import des données test accident: data & target
X_test = pd.read_csv('X_test.zip', nrows=100000)
display(X_test)

y_test = np.array(pd.read_csv('y_test.zip', nrows=100000)).ravel()
display(y_test)

,place,catu,sexe,trajet,locp,actp,etatp,cat_age,secu0,secu1,secu2,secu3,secu4,secu5,secu6,secu7,secu8,secu9,catv,obs,obsm,choc,manv,an,mois,lum,agg,int,atm,col,weekday,hr,catr,circ,vosp,prof,plan,surf,infra,situ
0,2.0,2,1,0.0,0.0,0,0.0,2,0,1,0,0,0,0,0,0,0,0,14.0,2.0,0.0,8.0,1.0,6,8,3,1,1,1.0,7.0,3,17,3.0,2.0,0.0,1.0,2.0,1.0,0.0,3.0
1,2.0,2,1,0.0,0.0,0,0.0,2,0,1,0,0,0,0,0,0,0,0,7.0,4.0,9.0,3.0,1.0,8,10,3,1,1,1.0,6.0,5,11,3.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0
2,1.0,3,2,2.0,1.0,5,1.0,1,0,0,0,0,0,0,0,0,1,0,7.0,0.0,1.0,8.0,1.0,9,9,2,2,1,1.0,6.0,3,17,3.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
3,1.0,1,2,5.0,0.0,0,0.0,5,0,1,0,0,0,0,0,0,0,0,7.0,0.0,0.0,0.0,0.0,16,9,1,1,1,1.0,4.0,3,11,4.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
4,1.0,1,2,0.0,0.0,0,0.0,4,0,1,0,0,0,0,0,0,0,0,7.0,0.0,2.0,1.0,1.0,9,9,5,2,2,1.0,1.0,1,17,3.0,2.0,0.0,1.0,1.0,1.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1.0,1,1,5.0,-1.0,0,-1.0,7,0,1,0,0,0,0,0,0,1,0,7.0,0.0,2.0,3.0,9.0,2020,12,1,2,6,8.0,3.0,6,11,4.0,2.0,0.0,1.0,1.0,1.0,5.0,1.0
999996,1.0,1,1,3.0,0.0,0,0.0,7,0,1,0,0,0,0,0,0,0,0,7.0,0.0,2.0,3.0,9.0,13,3,1,2,1,1.0,3.0,5,11,3.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0
999997,2.0,2,1,5.0,0.0,0,0.0,1,0,1,0,0,0,0,0,0,0,0,7.0,0.0,2.0,1.0,1.0,13,10,1,1,3,1.0,3.0,6,11,2.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0
999998,2.0,2,2,0.0,0.0,0,0.0,7,0,1,0,0,0,0,0,0,0,0,7.0,0.0,0.0,3.0,1.0,13,10,1,2,2,2.0,3.0,0,17,4.0,2.0,2.0,1.0,1.0,2.0,4.0,1.0


array([2, 2, 2, ..., 2, 2, 1], dtype=int64)

,place,catu,sexe,trajet,locp,actp,etatp,cat_age,secu0,secu1,secu2,secu3,secu4,secu5,secu6,secu7,secu8,secu9,catv,obs,obsm,choc,manv,an,mois,lum,agg,int,atm,col,weekday,hr,catr,circ,vosp,prof,plan,surf,infra,situ
0,1.0,1,2,3.0,0.0,0,0.0,6,0,1,0,0,0,0,0,0,0,0,7.0,0.0,0.0,4.0,2.0,8,4,1,2,1,3.0,2.0,1,11,2.0,1.0,0.0,1.0,1.0,3.0,0.0,1.0
1,1.0,1,1,0.0,0.0,0,0.0,3,0,1,0,0,0,0,0,0,0,0,7.0,0.0,1.0,1.0,1.0,9,12,5,2,2,1.0,6.0,6,17,4.0,2.0,3.0,1.0,1.0,1.0,0.0,1.0
2,1.0,1,1,0.0,0.0,0,0.0,4,0,1,0,0,0,0,0,0,0,0,7.0,2.0,0.0,3.0,0.0,12,3,5,2,1,1.0,7.0,2,7,3.0,3.0,0.0,2.0,3.0,1.0,0.0,3.0
3,1.0,1,2,0.0,-1.0,0,-1.0,6,0,1,0,0,0,1,0,0,0,0,7.0,0.0,2.0,3.0,2.0,2021,5,2,2,2,1.0,3.0,3,21,4.0,3.0,0.0,1.0,1.0,1.0,5.0,8.0
4,1.0,1,1,5.0,0.0,0,0.0,2,0,0,1,0,0,0,0,0,0,0,30.0,0.0,2.0,3.0,3.0,7,6,1,2,3,1.0,6.0,2,11,4.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500818,1.0,1,2,1.0,0.0,0,0.0,5,0,1,0,0,0,0,0,0,0,0,7.0,0.0,2.0,1.0,2.0,10,9,1,2,1,1.0,4.0,2,17,4.0,1.0,3.0,1.0,1.0,1.0,0.0,1.0
500819,1.0,1,1,0.0,0.0,0,0.0,6,0,1,0,0,0,0,0,0,0,0,7.0,0.0,0.0,0.0,1.0,12,7,1,2,1,1.0,4.0,3,11,3.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0
500820,1.0,1,1,4.0,-1.0,0,-1.0,6,1,1,0,0,0,0,0,0,0,0,7.0,0.0,2.0,3.0,11.0,2020,10,1,2,1,1.0,3.0,0,17,2.0,3.0,0.0,1.0,1.0,1.0,0.0,1.0
500821,1.0,3,2,5.0,5.0,1,0.0,2,0,0,0,0,0,0,0,0,1,0,7.0,0.0,2.0,3.0,1.0,10,10,1,2,1,1.0,2.0,5,11,4.0,2.0,0.0,1.0,1.0,2.0,0.0,1.0


array([2, 1, 2, ..., 1, 2, 2], dtype=int64)

# Modèle AdaBoostClassifier avec optimisation GridSearchCV

In [4]:
import time
# Début du chronomètre
start_time = time.time()

# Recherche des best parameters du GridSearchCV():

# Création du classificateur et construction du modèle sur les données d'entraînement

adaboost_model = AdaBoostClassifier()


# Définir les hyperparamètres à optimiser et leurs valeurs à tester
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.1, 0.5, 1.0],
    'estimator': [None, DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=3)],
    'algorithm': ['SAMME', 'SAMME.R'],
    'random_state': [42]
}
# Création de l'objet GridSearchCV avec le modèle et les hyperparamètres
grid_search = GridSearchCV(estimator=adaboost_model, param_grid=param_grid, cv=5)

# Exécution de la recherche sur grille pour trouver les meilleurs paramètres
grid_search.fit(X_sm, y_sm)

# Afficher les meilleurs paramètres et le meilleur score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score:", grid_search.best_score_)
print("\n The best estimator across ALL searched params:\n", grid_search.best_estimator_)

# Enregistrement du modèle (à faire après l'entraînement)
dump(grid_search, 'AdaBoostClassifier_BestParams.joblib') 

# Fin du chronomètre
end_time = time.time()

# Calcul de la durée d'exécution
execution_time = end_time - start_time

# Affichage du temps d'exécution
print("\nTemps d'exécution :", execution_time/60, "minutes")

Meilleurs paramètres: {'algorithm': 'SAMME.R', 'estimator': DecisionTreeClassifier(max_depth=3), 'learning_rate': 1.0, 'n_estimators': 200, 'random_state': 42}
Meilleur score: 0.8108409999999999

 The best estimator across ALL searched params:
 AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=3),
                   n_estimators=200, random_state=42)

Temps d'exécution : 589.0843814214071 minutes


In [6]:
# Maintenant, on applique le modèle entraîné et optimisé en termes d'hyper paramètre à l'ensemble de test
import time
# Début du chronomètre
start_time = time.time()


# Chargement du modèle entrainé optimisé par GridSearch() (à faire sur l'app Streamlit)
grid_search = load('AdaBoostClassifier_BestParams.joblib') 

# Application du modèle aux données de tests
y_pred = grid_search.predict(X_test)

print('Score de bonne prédiction:', grid_search.score(X_test, y_test))
print('\nCrosstab:\n', pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))

from imblearn.metrics import classification_report_imbalanced, geometric_mean_score

print('\nclassification_report_imbalanced:\n', classification_report_imbalanced(y_test, y_pred))

# Fin du chronomètre
end_time = time.time()

# Calcul de la durée d'exécution
execution_time = end_time - start_time

# Affichage du temps d'exécution
print("Temps d'exécution :", execution_time/60, "minutes")

Score de bonne prédiction: 0.8103701307647612

Crosstab:
 Classe prédite       1       2
Classe réelle                 
1               158333   46442
2                48529  247519

classification_report_imbalanced:
                    pre       rec       spe        f1       geo       iba       sup

          1       0.77      0.77      0.84      0.77      0.80      0.64    204775
          2       0.84      0.84      0.77      0.84      0.80      0.65    296048

avg / total       0.81      0.81      0.80      0.81      0.80      0.65    500823

Temps d'exécution : 0.7119779348373413 minutes


In [7]:
# Affiche des 5 variables les plus importantes

feats = {}
for feature, importance in zip(X_test.columns, grid_search.best_estimator_.feature_importances_):
    feats[feature] = importance 
    
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Importance'})
importances.sort_values(by='Importance', ascending=False).head(5)

,Importance
catv,0.119770
col,0.069990
manv,0.064584
obs,0.058000
obsm,0.056651


# Modèle AdaBoostClassifier sans optimisation GridSearchCV()

In [8]:
import time
# Début du chronomètre
start_time = time.time()

# Création du classificateur et entraînement du modèle sur les données de training
adaboost_model = AdaBoostClassifier(estimator = DecisionTreeClassifier(max_depth=3),
                                    learning_rate = 1,
                                    n_estimators = 200,
                                    algorithm = 'SAMME.R',
                                    random_state = 42
                                   )

# Entraîner le modèle AdaBoost
adaboost_model.fit(X_sm, y_sm)


# Enregistrement du modèle (à faire après l'entraînement)
dump(adaboost_model, 'AdaBoostClassifier_Params.joblib') 

print('Score de bonne prédiction sur dataset de training:', adaboost_model.score(X_sm, y_sm))

# Fin du chronomètre
end_time = time.time()

# Calcul de la durée d'exécution
execution_time = end_time - start_time

# Affichage du temps d'exécution
print("Temps d'exécution :", execution_time/60, "minutes")

Score de bonne prédiction sur dataset de training: 0.811884
Temps d'exécution : 7.516582389672597 minutes


In [9]:
# Obtenir les hyperparamètres du modèle
hyperparameters = adaboost_model.get_params()

# Afficher les hyperparamètres
for param, value in hyperparameters.items():
    print(param, ":", value)

algorithm : SAMME.R
base_estimator : deprecated
estimator__ccp_alpha : 0.0
estimator__class_weight : None
estimator__criterion : gini
estimator__max_depth : 3
estimator__max_features : None
estimator__max_leaf_nodes : None
estimator__min_impurity_decrease : 0.0
estimator__min_samples_leaf : 1
estimator__min_samples_split : 2
estimator__min_weight_fraction_leaf : 0.0
estimator__random_state : None
estimator__splitter : best
estimator : DecisionTreeClassifier(max_depth=3)
learning_rate : 1
n_estimators : 200
random_state : 42


In [11]:
import time
# Début du chronomètre
start_time = time.time()


# Chargement du modèle (à faire sur l'app Streamlit)
adaboost_model = load('AdaBoostClassifier_Params.joblib')

# Application du modèle aux données de tests
y_pred = adaboost_model.predict(X_test)

print('Score de bonne prédiction:', adaboost_model.score(X_test, y_test))
print('\nCrosstab:\n', pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))

from imblearn.metrics import classification_report_imbalanced, geometric_mean_score

print('\nclassification_report_imbalanced:\n', classification_report_imbalanced(y_test, y_pred))

# Fin du chronomètre
end_time = time.time()

# Calcul de la durée d'exécution
execution_time = end_time - start_time

# Affichage du temps d'exécution
print("Temps d'exécution :", execution_time/60, "minutes")

Score de bonne prédiction: 0.8103701307647612

Crosstab:
 Classe prédite       1       2
Classe réelle                 
1               158333   46442
2                48529  247519

classification_report_imbalanced:
                    pre       rec       spe        f1       geo       iba       sup

          1       0.77      0.77      0.84      0.77      0.80      0.64    204775
          2       0.84      0.84      0.77      0.84      0.80      0.65    296048

avg / total       0.81      0.81      0.80      0.81      0.80      0.65    500823

Temps d'exécution : 0.7473030726114909 minutes


In [12]:
# Affiche des 5 variables les plus importantes

feats = {}
for feature, importance in zip(X_test.columns, adaboost_model.feature_importances_):
    feats[feature] = importance 
    
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Importance'})
importances.sort_values(by='Importance', ascending=False).head(5)


,Importance
catv,0.119770
col,0.069990
manv,0.064584
obs,0.058000
obsm,0.056651
